In [82]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import time

## The Latest Mars News

In [83]:
executable_path = {"executable_path" : "chromedriver.exe"}
browser = Browser("chrome", **executable_path, headless=False)


In [84]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

html = browser.html
soup = bs(html, 'html.parser')



In [85]:
news_title = soup.find("div", class_="content_title").text
print(news_title)
news_p = soup.find("div", class_="article_teaser_body").text
print(news_p)


NASA Invites Public to Submit Names to Fly Aboard Next Mars Rover
From now till Sept. 30, the public can submit names to be stenciled on chips that will fly on the Mars 2020 rover and receive a souvenir boarding pass.


## Mars Featured Image

In [86]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

html = browser.html
soup = bs(html, 'html.parser')

In [88]:
# Retrieve full size image url
article = soup.find("li",class_="slide")
featured_image_url=article.find("img",class_='thumb')['src']
# Get full url
featured_image_url = url[:24] + featured_image_url
# Displace featured image url
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23222-640x350.jpg'

## Mars Weather

In [89]:
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)
soup = bs(response.text, 'lxml')

In [91]:
weather_list = soup.find_all("p",class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")
# Make sure the data we get is valid and complete which should include with "high" "low" "gusting"
for weather in weather_list:
    try:
        mars_weather = weather.text
        mars_weather_1 = [i.lower() for i in mars_weather.split(" ")]
        if ("low" in mars_weather_1) and ("high" in mars_weather_1) and ("gusting" in mars_weather_1):
            if mars_weather_1[-1].split(".")[0]=="hpapic" :
                mars_weather = mars_weather[:-26]
            print("The Latest Mars weather:")
            print(mars_weather)
            break
           
    except:
        pass   



The Latest Mars weather:
InSight sol 170 (2019-05-20) low -100.9ºC (-149.6ºF) high -21.2ºC (-6.1ºF)
winds from the SW at 4.7 m/s (10.5 mph) gusting to 14.6 m/s (32.6 mph)
pressure at 7.50 hPa


## Get Mars Facts

In [92]:
# Use Panda's `read_html` to parse the url
url="https://space-facts.com/mars/"
tables = pd.read_html(url)
Mars_df = tables[0]

Mars_df 


,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [93]:
# change columns name
Mars_df = Mars_df.rename(columns={0:"Description",1:"Value"})
Mars_df.set_index("Description",inplace=True)

Mars_df 


,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [94]:
# Convert dataframe to html table
mars_facts_html = Mars_df.to_html().replace("\n","")
#display(mars_facts_html)
mars_facts_html

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

## Hemisphere Images

In [95]:
url= "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
response= requests.get(url)

In [96]:
soup = bs(response.text,"html.parser")

In [97]:
# Find the hemisphere name
Hemisphere_name_list = []
#Get the url links to the four hemispheres
itemLink = soup.find_all("div",class_ = "description")
for item in itemLink:
    Hemisphere_name = item.find("h3").text
    Hemisphere_name = "_".join(Hemisphere_name.split(" "))
    Hemisphere_name_list.append(Hemisphere_name)
    
# Displace the four hemisphere name 
Hemisphere_name_list

['Cerberus_Hemisphere_Enhanced',
 'Schiaparelli_Hemisphere_Enhanced',
 'Syrtis_Major_Hemisphere_Enhanced',
 'Valles_Marineris_Hemisphere_Enhanced']

In [98]:
# Find the url linker
url_list=[]
itemLink_1 = soup.find_all("a",class_ = "itemLink product-item")
for item in itemLink_1:
    hemis_url=item['href']
    # Get full url
    hemis_url = url[:29]+ hemis_url
    url_list.append(hemis_url)

url_list

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [99]:
# Scrape the hemisphere urls
image_url_list=[]
for url in url_list:
    response = requests.get(url)
    soup = bs(response.text,"html.parser")
    image_url = soup.find("img",class_="wide-image")['src']
    image_url = url[:29] + image_url
    image_url_list.append(image_url)
image_url_list

['https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg']

In [100]:
# Store four hemisphere names and image urls in a list
hemisphere_image_urls =[]
# Store one pair of hemisphere name and image url in a dictionary
hemisphere_image_urls_dic={}

for i in range(len(image_url_list)):
    hemisphere_image_urls_dic["title"] = Hemisphere_name_list[i]
    hemisphere_image_urls_dic["img_url"] = image_url_list[i]
    hemisphere_image_urls.append({"title":Hemisphere_name_list[i], "img_url": image_url_list[i]})
hemisphere_image_urls

[{'title': 'Cerberus_Hemisphere_Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli_Hemisphere_Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis_Major_Hemisphere_Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles_Marineris_Hemisphere_Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]

In [101]:
# Close all browsers if still active
browser.quit()